In [66]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [67]:
backend = service.get_backend('ibm_nairobi')

In [68]:
sampler = Sampler(backend)

In [69]:
def create_tomo_circuit(basis, initial_layout, backend):
    '''
    create the quantum circuit of the cabello state and measure in the chosen basis
    the basis is a 4-element vector containing 0s, 1s and 2s 
    from [0, 0, 0, 0] meaning XXXX to [2, 2, 2, 2] meaning ZZZZ
    '''
    # the cabello circuit creating the state 2|1100>+2|0011>-|1010>-|0101>-|1001>-|0110>
    istvan_circ = QuantumCircuit(4, 4)
    istvan_circ.h(0)
    istvan_circ.h(1)
    istvan_circ.h(2)
    istvan_circ.rz(-0.73304, 0)
    istvan_circ.rz(2.67908, 2)
    istvan_circ.cnot(2, 0)
    istvan_circ.ry(-2.67908, 0)
    istvan_circ.h(2)
    istvan_circ.cnot(1, 0)
    istvan_circ.cnot(2, 3)
    istvan_circ.rz(1.5708, 2)
    istvan_circ.cnot(1, 3)
    istvan_circ.cnot(0, 2)
    # measurement. result '0' in the X basis means state |+>, in the Y basis |+i> and in the Z basis |0>
    for i in range(4):
        if basis[i] == 0:
            istvan_circ.h(i)
        elif basis[i] == 1:
            istvan_circ.sdg(i)
            istvan_circ.h(i)
    return transpile(istvan_circ, backend=backend, initial_layout=initial_layout)

In [70]:
# the bases we need to measure the state in, 81=3^4 in total
# the bases go from XXXX to ZZZZ in lexicographical order, just like the results in counts.txt
bases = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                bases.append([i, j, k, l])

In [71]:
layouts = []
for i in range(7):
    for j in range(7):
        for k in range(7):
            for l in range(7):
                if i != j and i != k and i != l and j != k and j != l and k != l:
                    layouts.append([i,j,k,l])

In [72]:
def error_circuits(layout):
    circs = []
    n = 7 # number of qubits of the device
    for qubit in layout:
        circs.append(QuantumCircuit(n, n))
        circs[-1].x(qubit)
        circs[-1].measure(range(n), range(n))
    return circs

In [73]:
all_res = []
for ind, layout in enumerate(layouts):
    # cycles over all possible qubit layouts and runs the circuit if it needs less than or equal to 11 CNOTs
    if not create_tomo_circuit([3,3,3,3], layout, backend).count_ops().get('cx', 0)>11:
        circs = []
        for basis in bases:
            circs.append(create_tomo_circuit(basis, layout, backend))
        #circs.extend(error_circuits(layout))
        res = sampler.run(circs, resilience_level=0, optimization_level=0)
        all_res.append(res)

In [75]:
len(all_res)

64

In [63]:
circ = QuantumCircuit(1, 1)
circ.x(0)
circ.measure(0, 0)

In [65]:
res2 = sampler.run(circ).result()

Traceback (most recent call last):
  Cell In[65], line 1
    res2 = sampler.run(circ).result()
  File /opt/conda/lib/python3.10/site-packages/qiskit_ibm_runtime/runtime_job.py:226 in result
    raise RuntimeJobFailureError(
RuntimeJobFailureError: 'Unable to retrieve job result. Job chufp6ai3durlgrlo5v0 has failed:\n2023-06-04T21:04:29.196985424Z Setting up watches.\n2023-06-04T21:04:29.196985424Z Watches established.\n2023-06-04T21:04:30.789593523Z INFO:     Started server process [7]\n2023-06-04T21:04:30.789734812Z INFO:     Waiting for application startup.\n2023-06-04T21:04:30.790017779Z INFO:     Application startup complete.\n2023-06-04T21:04:30.791453165Z INFO:     Uvicorn running on http://127.0.0.1:8081 (Press CTRL+C to quit)\n2023-06-04T21:05:31.190709402Z INFO:     127.0.0.1:39470 - "POST /run HTTP/1.1" 202 Accepted\n2023-06-04T21:05:33.406519267Z programruntime.runtime_backend - ERROR Job compilation status is error for job sampler_chufp6nnhfna4d6c2os0_3edd_0: \n2023-06-04T2